In [1]:
import json
with open("outputs/all_triples.json", "r", encoding="utf-8") as f:
    all_triples = json.load(f)


In [3]:
from neo4j import GraphDatabase

# ---------- 1. Configuration de la connexion Neo4j ----------
# Remplacez ces valeurs par celles de votre instance Neo4j
uri  = "bolt://localhost:7687"
user = "neo4j"
pwd  = "BNEOsucks8921_"

# Crée un driver Neo4j
driver = GraphDatabase.driver(uri, auth=(user, pwd))


# ---------- 2. Fonction d’insertion d’un triplet dans Neo4j ----------
def insert_triplet(tx, source, relation, target):
    """
    Crée ou récupère deux nœuds (Entity) nommés 'source' et 'target', 
    puis crée (si nécessaire) la relation entre eux.
    Le type de relation est stocké dans la propriété 'type' de l'arc.

    Exemple Cypher généré :
      MERGE (a:Entity {name: $source})
      MERGE (b:Entity {name: $target})
      MERGE (a)-[:RELATION {type: $relation}]->(b)
    """
    query = """
    MERGE (a:Source {name: $source})
    MERGE (b:Target {name: $target})
    MERGE (a)-[:RELATION {type: $relation}]->(b)
    """
    tx.run(query, source=source, relation=relation, target=target)


# ---------- 3. Fonction de chargement de tous les triplets ----------
def load_all_triplets(triplets):
    """
    Parcourt la liste 'triplets' (liste de tuples (source, relation, target))
    et exécute 'insert_triplet' pour chacun d’entre eux dans une même session.
    """
    with driver.session() as session:
        for src, rel, tgt in triplets:
            session.write_transaction(insert_triplet, src, rel, tgt)
    print(f"✅ {len(triplets)} triplets insérés dans Neo4j.")


# ---------- 4. Point d’entrée du script ----------
if __name__ == "__main__":
    # Supposons que vous ayez, dans un autre module, votre liste 'all_triples'
    # Exemple : all_triples = [("Navette autonome", "testée dans", "Toulouse"), ...]
    # Il faut donc importer ou recréer cette liste ici.
    #
    # Si votre extraction a produit un module Python ou un fichier pickle,
    # récupérez la liste de tuples et assignez-la à 'all_triples'.
    #
    # Par exemple, si vous avez sauvegardé vos relations dans un fichier JSON :
    #
    # import json
    # with open("outputs/all_triples.json", "r", encoding="utf-8") as f:
    #     all_triples = json.load(f)
    #
    # Mais ici, illustrons un exemple statique (à remplacer par votre propre liste) :

    # Charge tous les triplets dans Neo4j
    load_all_triplets(all_triples)

    # Ferme le driver une fois terminé
    driver.close()


C:\Users\maadt\AppData\Local\Temp\ipykernel_3636\2666090052.py:41: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_triplet, src, rel, tgt)


✅ 26746 triplets insérés dans Neo4j.


In [4]:
def fetch_all_triples():
    """
    Récupère l'ensemble des triplets (entité, relation, entité) dans Neo4j.
    """
    query = """
    MATCH (a:Source)-[r:RELATION]->(b:Target)
    RETURN a.name AS source, r.type AS relation, b.name AS target
    """
    triples = []
    with driver.session() as session:
        for record in session.run(query):
            triples.append({
                "source": record["source"],
                "relation": record["relation"],
                "target": record["target"]
            })
    return triples

# Exemple :
all_triples = fetch_all_triples()

C:\Users\maadt\AppData\Local\Temp\ipykernel_3636\1368597255.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [4]:
from llama_index.core.service_context_elements.llm_predictor import LLMPredictor
from llama_index.core import PromptHelper, ServiceContext
from llama_index.cli.rag.base import LLM
from llama_index.core.indices import KnowledgeGraphIndex
#from llama_index.indices.knowledge_graph.schema import KGTable
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from dotenv import load_dotenv
from llama_index.core.indices import PropertyGraphIndex
import os
import requests
from neo4j import GraphDatabase
import openai
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from pydantic import BaseModel




In [17]:
from dotenv import load_dotenv
load_dotenv()


True

In [9]:
from mistralai.client import MistralClient
from langchain_core.language_models import LLM
from typing import Optional, List, Mapping, Any
from pydantic import BaseModel, PrivateAttr
import os

class MistralLangChainLLM(LLM, BaseModel):
    """
    Wrapper LangChain-compatible pour le SDK mistralai.
    """

    temperature: float = 0.2
    model_name: str = "mistral-small"
    
    # ✅ Ajout d'attributs privés compatibles Pydantic
    _api_key: str = PrivateAttr()
    _client: MistralClient = PrivateAttr()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._api_key = os.getenv("MISTRAL_API_KEY")
        if not self._api_key:
            raise ValueError("⚠️ La clé API MISTRAL_API_KEY est manquante dans l’environnement.")
        self._client = MistralClient(api_key=self._api_key)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        messages = [{"role": "user", "content": prompt}]
        response = self._client.chat(
            model=self.model_name,
            messages=messages,
            temperature=self.temperature,
        )
        return response.choices[0].message.content

    @property
    def _llm_type(self) -> str:
        return "mistral"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
        }


In [6]:


import os
from typing import List, Any
from neo4j import GraphDatabase
from langchain.schema import BaseRetriever, Document
from pydantic import PrivateAttr


class Neo4jRetriever(BaseRetriever):
    """
    Retriever LangChain qui interroge Neo4j pour ramener un sous-graphe pertinent.
    """

    # Déclare driver comme attribut privé afin que Pydantic ne l'exige pas comme champ
    _driver: Any = PrivateAttr()

    def __init__(self):
        # Appelle le constructeur de BaseModel
        super().__init__()

        uri = os.getenv("NEO4J_URI")
        user = os.getenv("NEO4J_USER")
        pwd  = os.getenv("NEO4J_PWD")
        self._driver = GraphDatabase.driver(uri, auth=(user, pwd))

    def get_relevant_documents(self, query: str) -> List[Document]:
        """
        1) On extrait des tokens (mots) de la question,
        2) On interroge Neo4j pour chaque token correspondant
           à un nœud Entity.name,
        3) On construit un Document par relation trouvée.
        """
        # Tokenisation basique ; dans la vraie vie, on ferait un NER ou des lowercase+strip
        tokens = [tok.strip() for tok in query.split() if len(tok) > 1]
        seen_relations = set()
        docs: List[Document] = []

        with self._driver.session() as session:
            for tok in tokens:
                cypher = """
                MATCH (n:Entity {name: $name})-[r]-(m:Entity)
                RETURN n.name AS source, type(r) AS rel, m.name AS target
                """
                result = session.run(cypher, name=tok)
                for record in result:
                    src = record["source"]
                    rel = record["rel"]
                    tgt = record["target"]
                    triple_text = f"{src} {rel} {tgt}."
                    if triple_text not in seen_relations:
                        seen_relations.add(triple_text)
                        docs.append(Document(page_content=triple_text))

        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # Pour la plupart des usages on peut renvoyer synchrone
        return self.get_relevant_documents(query)

    def __del__(self):
        try:
            self._driver.close()
        except:
            pass


C:\Users\maadt\AppData\Local\Temp\ipykernel_17420\3120548857.py:8: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class Neo4jRetriever(BaseRetriever):
C:\Users\maadt\AppData\Local\Temp\ipykernel_17420\3120548857.py:8: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class Neo4jRetriever(BaseRetriever):


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import RetrievalQA
import dotenv
from Retr

# ----------------------------------------
# 1. Charger les variables d’environnement
# ----------------------------------------
load_dotenv()

# ----------------------------------------
# 2. Instancier le LLM Albert
# ----------------------------------------
llm = MistralLangChainLLM(temperature=0.2)

# ----------------------------------------
# 3. Créer le prompt template pour la QA
# ----------------------------------------
# {context} = textes du graphe Neo4j
# {question} = question utilisateur
prompt_template = """
Tu es un assistant expert en véhicules autonomes.
Voici le contexte extrait d'un graphe de connaissances :
{context}

Question : {question}

Réponds de façon précise, en t’appuyant seulement sur ces faits. Si ce n’est pas dans le contexte, répond « Désolé, je n’ai pas cette information. ».
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# ----------------------------------------
# 4. Instancier le retriever Neo4j personnalisé
# ----------------------------------------
graph_retriever = Neo4jRetriever()

# ----------------------------------------
# 5. Construire la chaîne RetrievalQA
# ----------------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",           # on bourre tout le contexte d’un coup
    retriever=graph_retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)

# ----------------------------------------
# 6. Boucle interactive
# ----------------------------------------
if __name__ == "__main__":
    print("=== Chat GraphRAG (Neo4j + Albert via LangChain) ===")
    while True:
        question = input("\nPose ta question (ou « exit » pour quitter) : ")
        if question.lower().strip() in ("exit", "quit"):
            break

        # LangChain :
        #  1) graph_retriever.get_relevant_documents(question) → liste de Docs
        #  2) Concatène “context” = sommaire des docs, plus “question” dans le prompt
        #  3) Envoie tout à llm._call(prompt_final) → Albert → génération
        result = qa_chain.invoke({"query": question})
        print("\n📝 Réponse :")
        print(result["result"])

=== Chat GraphRAG (Neo4j + Albert via LangChain) ===

📝 Réponse :
L'automobile est liée à la mobilité, ce qui signifie qu'elle joue un rôle important dans le domaine des déplacements. Elle est également associée à la France, indiquant que la France a une relation avec l'industrie automobile, que ce soit en termes de production, de consommation ou de réglementation. En outre, l'automobile a également une relation avec la population, ce qui suggère que les voitures sont un moyen de transport important pour les gens. Cependant, ce contexte ne fournit pas d'informations spécifiques sur le fonctionnement, les caractéristiques ou les types d'automobiles.
